<a href="https://colab.research.google.com/github/njgeorge000158/Aviation-Accidents-Analysis-Project-v1/blob/main/AlphabetSoupCharityOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1239]:
#*******************************************************************************************
 #
 #  File Name:  AlphabetSoupCharityOptimization.ipynb
 #
 #  File Description:
 #      This interactive Python notebook,AlphabetSoupCharityOptimization.ipynb, reads
 #      a csv file, LendingData.csv, in the Resources folder, and uses Python and neural
 #      networks methods to use the features in the provided dataset to create a binary
 #      classifier that can predict whether applicants for charity donations will be
 #      successful if funded by the nonprofit foundation, Alphabet Soup.
 #
 #      I reinstalled the scikit-learn module using the following commands to allow the
 #      RandomOverSampler function to work:
 #
 #      pip3 uninstall scikit-learn
 #      pip3 install scikit-learn==1.2.2
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  12/03/2023      Initial Development                     N. James George
 #
 #******************************************************************************************/

from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.insert(0,'/content/drive/My Drive/deep-learning-challenge')

!pip install -q dataframe_image
!pip install aspose-words
!pip install hvplot
!pip install silence-tensorflow
!pip install keras-tuner

import PyConstants as constant
import PyFunctions as function
import PyLogConstants as log_constant
import PyLogFunctions as log_function
import PyLogSubRoutines as log_subroutine
import PySubRoutines as subroutine

import AlphabetSoupCharityOptimizationSearchFunctions as local_function

import pandas as pd

import json

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1240]:
CONSTANT_LOCAL_FILE_NAME \
    = 'AlphabetSoupCharityOptimization.ipynb'

CONSTANT_CHARITY_DATA_CSV_FILE_PATH \
    = '/content/drive/My Drive/deep-learning-challenge/Resources/CharityData.csv'

CONSTANT_OPTIMIZATION_FILE_PATH \
    = '/content/drive/My Drive/deep-learning-challenge/Resources/AlphabetSoupCharityOptimization.keras'

log_subroutine \
    .BeginProgramExecution \
        ('AlphabetSoupCharityOptimization')

# <br> **Section 1: Data Acquistion**

## **1.1: Import and Read Charity Data**

In [1241]:
charityDataFrame \
    = function. \
        ReturnCSVFileAsDataFrame \
            (CONSTANT_CHARITY_DATA_CSV_FILE_PATH)


log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

## **1.2: Display Charity Data**

In [1242]:
captionString \
    = 'Table 1.2.1: Charity Data Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (charityDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
"10,520,599",BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,"5,000",1
"10,531,628",AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,"108,590",1
"10,547,893",ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0
"10,553,066",SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,"6,692",1
"10,556,103",GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,"142,590",1
"10,556,855",MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,1,0,N,"5,000",1
"10,558,440",FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,"31,452",1
"10,566,033",ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,"7,508,025",1
"10,570,430",ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,"94,389",1
"10,571,689",INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,"5,000",0


# <br> **Section 2: Preprocessing**

## **2.1: Drop the Non-Beneficial ID Columns**

In [1243]:
charityDataFrame \
    .drop \
        (['EIN', 'STATUS', 'SPECIAL_CONSIDERATIONS'],
         axis = 1,
         inplace = True)

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

In [1244]:
captionString \
    = 'Table 1.2.2: Modified Charity Data Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (charityDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,"5,000",1
AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,"108,590",1
ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,"5,000",0
SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,"6,692",1
GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,"142,590",1
MINORITY ORGAN & TISSUE TRANSPLANT & EDUCATION PROGRAM OF TENNESSEE,T3,Independent,C1200,Preservation,Trust,0,"5,000",1
FRIENDS OF ARTS COUNCIL OF GREATER DENHAM SPRINGS INC,T3,Independent,C1000,Preservation,Trust,100000-499999,"31,452",1
ISRAEL EMERGENCY ALLIANCE,T3,Independent,C2000,Preservation,Trust,10M-50M,"7,508,025",1
ARAMCO BRATS INC,T7,Independent,C1000,ProductDev,Trust,1-9999,"94,389",1
INTERNATIONAL ASSOCIATION OF FIRE FIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,"5,000",0


## **2.2: Determine the Number of Unique Values in Each Column**

In [1245]:
numberOfUniqueValuesInEachColumnSeries \
    = charityDataFrame \
        .nunique()

log_function \
    .DebugReturnObjectWriteObject \
        (numberOfUniqueValuesInEachColumnSeries)

In [1246]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(numberOfUniqueValuesInEachColumnSeries))

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64


## **2.3: NAME Binning**

### **Look at NAME Value Counts for Binning**

In [1247]:
nameValueCountsForBinningSeries \
    = charityDataFrame['NAME'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

nameValueCountsForBinningSeries \
    .name = 'NAME'

log_function \
    .DebugReturnObjectWriteObject \
        (nameValueCountsForBinningSeries)

### **Choose a Cutoff Value and Create a List of Names to be Replaced**

In [1248]:
nameCutoffInteger = 2

names_to_replace \
    = list \
        (nameValueCountsForBinningSeries \
            [nameValueCountsForBinningSeries < nameCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (names_to_replace)

### **Replace Application Types in DataFrame**

In [1249]:
for name in names_to_replace:

    charityDataFrame['NAME'] \
        = charityDataFrame['NAME'] \
            .replace \
                (name, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [1250]:
nameValueCountsForBinningSeries \
    = charityDataFrame['NAME'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

nameValueCountsForBinningSeries \
    .name = 'NAME'

log_function \
    .DebugReturnObjectWriteObject \
        (nameValueCountsForBinningSeries)

## **2.3: APPLICATION_TYPE Binning**

### **Look at APPLICATION_TYPE Value Counts for Binning**

In [1251]:
applicationTypeValueCountsForBinningSeries \
    = charityDataFrame['APPLICATION_TYPE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

applicationTypeValueCountsForBinningSeries \
    .name = 'APPLICATION_TYPE'

log_function \
    .DebugReturnObjectWriteObject \
        (applicationTypeValueCountsForBinningSeries)

In [1252]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(applicationTypeValueCountsForBinningSeries))

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


### **Choose a Cutoff Value and Create a List of Application Types to be Replaced**

In [1253]:
applicationTypeCutoffInteger = 157

application_types_to_replace \
    = list \
        (applicationTypeValueCountsForBinningSeries \
             [applicationTypeValueCountsForBinningSeries < applicationTypeCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (application_types_to_replace)

In [1254]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(application_types_to_replace))

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']


### **Replace Application Types in DataFrame**

In [1255]:
for applicationTypes in application_types_to_replace:

    charityDataFrame['APPLICATION_TYPE'] \
        = charityDataFrame['APPLICATION_TYPE'] \
            .replace \
                (applicationTypes, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [1256]:
applicationTypeValueCountsForBinningSeries \
    = charityDataFrame['APPLICATION_TYPE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

applicationTypeValueCountsForBinningSeries \
    .name = 'APPLICATION_TYPE'

log_function \
    .DebugReturnObjectWriteObject \
        (applicationTypeValueCountsForBinningSeries)

In [1257]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(applicationTypeValueCountsForBinningSeries))

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


## **2.4: CLASSIFICATION Binning**

### **Look at CLASSIFICATION Value Counts for Binning**

In [1258]:
classificationValueCountsForBinningSeries \
    = charityDataFrame['CLASSIFICATION'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

classificationValueCountsForBinningSeries \
    .name = 'CLASSIFICATION'

log_function \
    .DebugReturnObjectWriteObject \
        (classificationValueCountsForBinningSeries)

In [1259]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(classificationValueCountsForBinningSeries))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C2600        1
C4200        1
C2190        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


### **Look at CLASSIFICATION Value Counts > 1**

In [1260]:
classificationValueCountsForBinningSeries \
    [classificationValueCountsForBinningSeries > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

### **Choose a Cutoff Value and Create a List of Classifications to be Replaced**

In [1261]:
classificationCutoffInteger = 96

classifications_to_replace \
    = list \
        (classificationValueCountsForBinningSeries \
             [classificationValueCountsForBinningSeries < classificationCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (classifications_to_replace)

### **Replace Classifications in DataFrame**

In [1262]:
for classification in classifications_to_replace:

    charityDataFrame['CLASSIFICATION'] \
        = charityDataFrame['CLASSIFICATION'] \
            .replace \
                (classification, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [1263]:
classificationValueCountsForBinningSeries \
    = charityDataFrame['CLASSIFICATION'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

classificationValueCountsForBinningSeries \
    .name = 'CLASSIFICATION'

log_function \
    .DebugReturnObjectWriteObject \
        (classificationValueCountsForBinningSeries)

In [1264]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(classificationValueCountsForBinningSeries))

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64


## **2.5: USE_CASE Binning**

### **Look at USE_CASE Value Counts for Binning**

In [1265]:
useCaseValueCountsForBinningSeries \
    = charityDataFrame['USE_CASE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

useCaseValueCountsForBinningSeries \
    .name = 'USE_CASE'

log_function \
    .DebugReturnObjectWriteObject \
        (useCaseValueCountsForBinningSeries)

In [1266]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(useCaseValueCountsForBinningSeries))

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64


### **Choose a Cutoff Value and Create a List of Use Cases to be Replaced**

In [1267]:
useCaseCutoffInteger = 147

use_cases_to_replace \
    = list \
        (useCaseValueCountsForBinningSeries \
             [useCaseValueCountsForBinningSeries < useCaseCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (use_cases_to_replace)

### **Replace Use Cases in DataFrame**

In [1268]:
for useCase in use_cases_to_replace:

    charityDataFrame['USE_CASE'] \
        = charityDataFrame['USE_CASE'] \
            .replace \
                (useCase, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [1269]:
useCaseValueCountsForBinningSeries \
    = charityDataFrame['USE_CASE'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

useCaseValueCountsForBinningSeries \
    .name = 'USE_CASE'

log_function \
    .DebugReturnObjectWriteObject \
        (useCaseValueCountsForBinningSeries)

In [1270]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(useCaseValueCountsForBinningSeries))

Preservation     28095
ProductDev        5671
CommunityServ      384
Other              149
Name: USE_CASE, dtype: int64


## **2.6: ASK_AMT Binning**

### **Look at ASK_AMT Value Counts for Binning**

In [1271]:
askAmtValueCountsForBinningSeries \
    = charityDataFrame['ASK_AMT'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

askAmtValueCountsForBinningSeries \
    .name = 'ASK_AMT'

log_function \
    .DebugReturnObjectWriteObject \
        (askAmtValueCountsForBinningSeries)

In [1272]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(askAmtValueCountsForBinningSeries))

5000        25398
15583           3
63981           3
6725            3
10478           3
            ...  
11336           1
1270766         1
121619          1
46915           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


### **Choose a Cutoff Value and Create a List of Ask Amounts to be Replaced**

In [1273]:
askAmtCutoffInteger = 4

ask_amounts_to_replace \
    = list \
        (askAmtValueCountsForBinningSeries \
             [askAmtValueCountsForBinningSeries < askAmtCutoffInteger] \
          .index)

log_function \
    .DebugReturnObjectWriteObject \
        (ask_amounts_to_replace)

### **Replace Ask Amounts in DataFrame**

In [1274]:
for askAmt in ask_amounts_to_replace:

    charityDataFrame['ASK_AMT'] \
        = charityDataFrame['ASK_AMT'] \
            .replace \
                (askAmt, 'Other')

log_function \
    .DebugReturnObjectWriteObject \
        (charityDataFrame)

### **Check to Make Sure Binning was Successful**

In [1275]:
askAmtValueCountsForBinningSeries \
    = charityDataFrame['ASK_AMT'] \
        .value_counts() \
        .sort_values \
            (ascending = False)

askAmtValueCountsForBinningSeries \
    .name = 'ASK_AMT'

log_function \
    .DebugReturnObjectWriteObject \
        (askAmtValueCountsForBinningSeries)

In [1276]:
log_subroutine \
    .PrintAndLogWriteText \
        (str(askAmtValueCountsForBinningSeries))

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64


## **2.7: Final Preprocessing**

### **Convert Categorical Data to Numeric with `pd.get_dummies`**

In [1277]:
dummiesDataFrame \
    = pd.get_dummies \
        (charityDataFrame)

log_function \
    .DebugReturnObjectWriteObject \
        (dummiesDataFrame)

In [1278]:
captionString \
    = 'Table 2.5.1: Dummies DataFrame Table'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (dummiesDataFrame.head(12),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

### **Split Preprocessed Data into Features and Target Arrays**

In [1279]:
yNumpyArray \
    = dummiesDataFrame['IS_SUCCESSFUL'].values

log_function \
    .DebugReturnObjectWriteObject \
        (yNumpyArray)

In [1280]:
XNumpyArray \
    = dummiesDataFrame \
        .drop \
            (['IS_SUCCESSFUL'],
             axis = 1) \
        .values

log_function \
    .DebugReturnObjectWriteObject \
        (XNumpyArray)

### **Split the Preprocessed Data into a Training and Testing Dataset**

In [1281]:
XTrainNumpyArray, \
XTestNumpyArray, \
yTrainNumpyArray, \
yTestNumpyArray \
    = train_test_split \
            (XNumpyArray,
             yNumpyArray,
             random_state = 21)

In [1282]:
log_function \
    .DebugReturnObjectWriteObject \
        (XTrainNumpyArray)

In [1283]:
log_function \
    .DebugReturnObjectWriteObject \
        (XTestNumpyArray)

In [1284]:
log_function \
    .DebugReturnObjectWriteObject \
        (yTrainNumpyArray)

In [1285]:
log_function \
    .DebugReturnObjectWriteObject \
        (yTestNumpyArray)

### **Create a StandardScaler Instance**

In [1286]:
currentStandardScalar \
    = StandardScaler()

log_function \
    .DebugReturnObjectWriteObject \
        (currentStandardScalar)

### **Fit the StandardScaler**

In [1287]:
XStandardScalar \
    = currentStandardScalar \
        .fit \
            (XTrainNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XStandardScalar)

### **Scale the Data**

In [1288]:
XTrainScaledNumpyArray \
    = XStandardScalar \
        .transform \
            (XTrainNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XTrainScaledNumpyArray)

In [1289]:
XTestScaledNumpyArray \
    = XStandardScalar \
        .transform \
            (XTestNumpyArray)

log_function \
    .DebugReturnObjectWriteObject \
        (XTestScaledNumpyArray)

# <br> **Section 3: Compile, Train, Evaluate, and Export the Model**

## **3.1: Compile Model**

### **Model Definition**

In [1290]:
# These line of code define the model with the number of input features and hidden nodes for each layer.
neuralNetSequential \
    = tf.keras.models.Sequential()

log_function \
    .DebugReturnObjectWriteObject \
        (neuralNetSequential)

### **Layers**

In [1291]:
inputDimensionInteger \
    = len(XTrainScaledNumpyArray[0])

# This line of code is the first layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 58,
              activation = 'leaky_relu',
              input_dim = inputDimensionInteger))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 19,
              activation = 'leaky_relu'))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 97,
              activation = 'leaky_relu'))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 48,
              activation = 'leaky_relu'))

# This line of code is the hidden layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 7,
              activation = 'leaky_relu'))

# This line of code is the output layer.
neuralNetSequential \
    .add \
        (tf.keras.layers.Dense \
             (units = 1,
              activation = 'sigmoid'))

### **Check Model**

In [1292]:
neuralNetSequential.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_126 (Dense)           (None, 58)                48720     
                                                                 
 dense_127 (Dense)           (None, 19)                1121      
                                                                 
 dense_128 (Dense)           (None, 97)                1940      
                                                                 
 dense_129 (Dense)           (None, 48)                4704      
                                                                 
 dense_130 (Dense)           (None, 7)                 343       
                                                                 
 dense_131 (Dense)           (None, 1)                 8         
                                                                 
Total params: 56,836
Trainable params: 56,836
Non-tra

### **Compile**

In [1293]:
neuralNetSequential \
    .compile \
        (loss = 'binary_crossentropy',
         optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
         metrics = ['accuracy'])

## **3.2: Train Model**

In [1294]:
neuralNetSequential \
    .fit \
        (XTrainScaledNumpyArray,
         yTrainNumpyArray,
         epochs = 100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.4717 - accuracy: 0.7769
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4193 - accuracy: 0.8014
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4113 - accuracy: 0.8047
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4090 - accuracy: 0.8049
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4083 - accuracy: 0.8057
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4066 - accuracy: 0.8061
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4060 - accuracy: 0.8063
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4050 - accuracy: 0.8075
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4043 - accuracy: 0.8067
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4040 - accura

## **3.3: Evaluate Model**

In [1295]:
modelLossFloat, \
modelAccuracyFloat \
    = neuralNetSequential.evaluate \
        (XTestScaledNumpyArray,
         yTestNumpyArray,
         verbose = 2)

log_subroutine \
    .PrintAndLogWriteText \
        (f'\nModel Loss: {round(modelLossFloat * 100, 2)}%, '
         + f'Model Accuracy: {round(modelAccuracyFloat*100, 2)}%')

268/268 - 0s - loss: 0.4642 - accuracy: 0.8058 - 484ms/epoch - 2ms/step

Model Loss: 46.42%, Model Accuracy: 80.58%


## **3.4: Export Model**

In [1296]:
neuralNetSequential.save \
    (CONSTANT_OPTIMIZATION_FILE_PATH)

In [1297]:
#log_subroutine \
#    .EndProgramExecution()